# 0. Импорты


In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# os.chdir("/content/drive/MyDrive/bmstu/semester_3/NLP/hw_2")
# print("Текущая папка:", os.getcwd())

In [2]:
!git clone https://github.com/ruslann19/bmstu_fs12_nlp.git
!cp -r ./bmstu_fs12_nlp/hw/hw_2/src/ ./src/
!rm -rf ./bmstu_fs12_nlp/

Cloning into 'bmstu_fs12_nlp'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 73 (delta 24), reused 61 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (73/73), 39.74 MiB | 40.65 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [3]:
import os
import gdown


def download_dataset(file_id, output_path):
    url = f"https://drive.google.com/uc?id={file_id}"
    print("Скачивание данных...")
    gdown.download(url, output_path, quiet=False)


os.makedirs("./data/", exist_ok=True)
download_dataset(
    file_id="1ECxjo60HQwYqn1MdP5cukP1iYNw-GXxZ",
    output_path="./data/train.csv",
)
download_dataset(
    file_id="153iQ_jMqFsdHhwvhj4lgN8LeOkLUjhkl",
    output_path="./data/test.csv",
)

Скачивание данных...


Downloading...
From (original): https://drive.google.com/uc?id=1ECxjo60HQwYqn1MdP5cukP1iYNw-GXxZ
From (redirected): https://drive.google.com/uc?id=1ECxjo60HQwYqn1MdP5cukP1iYNw-GXxZ&confirm=t&uuid=d4f1dae9-ad1d-499f-b52e-d47c5ad728c6
To: /kaggle/working/data/train.csv
100%|██████████| 362M/362M [00:02<00:00, 127MB/s]  


Скачивание данных...


Downloading...
From: https://drive.google.com/uc?id=153iQ_jMqFsdHhwvhj4lgN8LeOkLUjhkl
To: /kaggle/working/data/test.csv
100%|██████████| 2.99M/2.99M [00:00<00:00, 83.9MB/s]


In [4]:
# Для Kaggle
!pip install --upgrade scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 70.4 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.2 which is incompatible.


In [5]:
from src.preprocessing import clean_text
from sklearn.model_selection import train_test_split
from src.bert_regressor import BertRegressor, BertRegressorConfig
from src import compute_metrics
from src import KeepBestModelCallback
from src.predict_from_dataset import predict_from_dataset

import numpy as np
import pandas as pd
import torch
from datasets import Dataset
import os
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    Trainer,
)

2025-11-26 13:57:59.946034: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764165480.118180      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764165480.167789      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

# 1. Подготовка данных


In [6]:
nrows = None

# Загрузка данных из CSV
print("Загрузка датасета...")
dataset_df = pd.read_csv(
    filepath_or_buffer="./data/train.csv",
    nrows=nrows,
)

dataset_df = dataset_df[["text", "rating"]]
dataset_df = dataset_df.rename(columns={"rating": "labels"})

# Выкинуть оценки, равные 0
dataset_df = dataset_df[dataset_df["labels"] != 0]

if nrows is None:
    # Оставить только 50_000 оценок, равных 5, для балансировки распределения оценок
    dataset_df_5 = dataset_df[dataset_df["labels"] == 5].sample(
        n=50_000,
        random_state=42,
    )
    dataset_df_other = dataset_df[dataset_df["labels"] != 5]
    dataset_df = pd.concat([dataset_df_5, dataset_df_other]).reset_index(
        drop=True
    )

print("Очистка текстов...")
dataset_df["text"] = dataset_df["text"].map(clean_text)


print("Разделение на train/val...")
if nrows is None:
    train_df, val_df = train_test_split(
        dataset_df,
        test_size=0.2,
        random_state=42,
        # Во всём датасете найдутся объекты, чтобы разделить их на train/val
        stratify=dataset_df["labels"],
    )
else:
    train_df, val_df = train_test_split(
        dataset_df,
        test_size=0.2,
        random_state=42,
    )

train_df = train_df.astype({"labels": "float"})
val_df = val_df.astype({"labels": "float"})

train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)

print("Готово")

Загрузка датасета...
Очистка текстов...
Разделение на train/val...
Готово


In [7]:
test_df = pd.read_csv("./data/test.csv")
test_df = test_df[["review_text"]]
test_df = test_df.rename(columns={"review_text": "text"})
test_df["text"] = test_df["text"].map(clean_text)

test_data = Dataset.from_pandas(test_df)

# 2. Загрузка токенизатора


In [8]:
hf_model_name = "d0rj/e5-base-en-ru"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=hf_model_name)


def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt",
    )


train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/471 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Map:   0%|          | 0/127428 [00:00<?, ? examples/s]

Map:   0%|          | 0/31857 [00:00<?, ? examples/s]

Map:   0%|          | 0/5069 [00:00<?, ? examples/s]

# 2. Кастомная модель: BERT + Head (Regressor)

In [10]:
classes_counts = train_df["labels"].astype(int).value_counts().to_dict()

config = BertRegressorConfig(
    hf_model_name=hf_model_name,
    loss_type="WeightedMSELoss",
    classes_counts=classes_counts,
)

model = BertRegressor(config)

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors:   0%|          | 0.00/529M [00:00<?, ?B/s]

# 3. Обучение


In [11]:
steps = 4_000

training_args = TrainingArguments(
    # Мониторинг
    output_dir="./checkpoints",
    overwrite_output_dir=True,
    save_strategy="steps",
    eval_strategy="steps",
    logging_strategy="steps",
    save_steps=steps,
    eval_steps=steps,
    logging_steps=steps,
    save_total_limit=2,
    report_to="none",
    # Обучение
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    # Оптимизация
    dataloader_pin_memory=torch.cuda.is_available(),
    # Выбрать лучшую модель в конце обучения
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
    callbacks=[
        KeepBestModelCallback(
            best_model_dir="./checkpoints/best_model",
            metric_name="eval_loss",
            greater_is_better=False,
        )
    ],
)

In [13]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss,Mse,Mae,Rmse,R2
4000,3.234800,2.863705,0.502161,0.502477,0.708633,0.782508
8000,2.694200,3.062276,0.504561,0.475895,0.710324,0.781469
12000,2.390200,2.716346,0.454341,0.431660,0.674048,0.803220
16000,2.432100,2.816515,0.487441,0.493341,0.698170,0.788884
20000,2.181700,2.648369,0.461535,0.478061,0.679364,0.800104
24000,2.135200,2.614284,0.445934,0.435533,0.667783,0.806861


config.json:   0%|          | 0.00/911 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

KeyboardInterrupt: 

# 4. Предсказание

In [14]:
predictions = predict_from_dataset("./checkpoints/best_model", test_data)

  0%|          | 0/634 [00:00<?, ?it/s]

In [15]:
predictions_df = pd.DataFrame(
    data={
        "id": np.arange(len(predictions)),
        "predicted_rating": np.clip(predictions, 1, 5),
    }
)

display(predictions_df)

,id,predicted_rating
0,0,2.763470
1,1,5.000000
2,2,1.000000
3,3,1.255858
4,4,3.862999
...,...,...
5064,5064,1.124282
5065,5065,3.679532
5066,5066,1.355152
5067,5067,1.037079


In [16]:
os.makedirs(name="./predictions", exist_ok=True)
predictions_df.to_csv(
    "./predictions/e5_base_linear.tsv",
    sep="\t",
    header=False,
    index=False,
)

In [17]:
!ls checkpoints

best_model  checkpoint-20000  checkpoint-24000


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
!cp -r checkpoints checkpoints_2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
